In [ ]:
%load_ext autoreload
%autoreload 2


import matplotlib.pyplot as plt
import seaborn as sns


import warnings
import numpy as np
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
%pylab inline


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf
import numpy as np

import tsgm

In [ ]:
seq_len = 256
feat_dim = 1
num_classes = 2
latent_dim = 32

In [ ]:
class cVAE_CONV5Architecture(object):
    def __init__(self, seq_len, feat_dim, latent_dim, output_dims=2):
        self._seq_len = seq_len
        self._feat_dim = feat_dim
        self._latent_dim = latent_dim
        self._output_dims = output_dims
        
        self._encoder = self._build_encoder()
        self._decoder = self._build_decoder()
        
    def _build_encoder(self):
        encoder_inputs = keras.Input(shape=(self._seq_len, self._feat_dim + self._output_dims))

        x = layers.Conv1D(64, 10, activation="relu", strides=1, padding="same")(encoder_inputs)
        x = layers.Dropout(rate=0.2)(x)
        x = layers.Conv1D(64, 2, activation="relu", strides=1, padding="same")(x)
        x = layers.Dropout(rate=0.2)(x)
        x = layers.Conv1D(64, 2, activation="relu", strides=1, padding="same")(x)
        x = layers.Dropout(rate=0.2)(x)
        x = layers.Conv1D(64, 2, activation="relu", strides=1, padding="same")(x)
        x = layers.Dropout(rate=0.2)(x)
        x = layers.Conv1D(64, 4, activation="relu", strides=1, padding="same")(x)
        x = layers.Dropout(rate=0.2)(x)
        x = layers.Flatten()(x)
        x = layers.Dense(512, activation="relu")(x)
        x = layers.Dense(64, activation="relu")(x)
        z_mean = layers.Dense(self._seq_len * self._latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(self._seq_len * self._latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
        return encoder
        
    def _build_decoder(self):
        inputs = keras.Input(shape=(self._seq_len, self._latent_dim + self._output_dims,))
        x = layers.Conv1DTranspose(64, 2, strides=2, padding="same")(inputs)
        x = layers.LeakyReLU(alpha=0.2)(x)        
        x = layers.Dropout(rate=0.2)(x)
        x = layers.Conv1DTranspose(64, 2, strides=2, padding="same")(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        x = layers.Dropout(rate=0.2)(x)
        x = layers.Conv1DTranspose(64, 2, strides=2, padding="same")(x)
        x = layers.LeakyReLU(alpha=0.2)(x)
        x = layers.Dropout(rate=0.2)(x)

        pool_and_stride = round((x.shape[1] + 1) / (seq_len + 1))
        x = layers.AveragePooling1D(pool_size=pool_and_stride, strides=pool_and_stride)(x)
        d_output = layers.LocallyConnected1D(1, 1, activation="sigmoid")(x)

        decoder = keras.Model(inputs, d_output, name="decoder")
        return decoder

In [ ]:
class cBetaVAE(keras.Model):
    def __init__(self, encoder, decoder, latent_dim, temporal, beta=1.0, **kwargs):
        super(cBetaVAE, self).__init__(**kwargs)
        self.beta = beta
        self.encoder = encoder
        self.decoder = decoder

        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")     
        self.latent_dim = latent_dim
        self._temporal = temporal

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def generate(self, labels):
        batch_size = tf.shape(labels)[0]
        z = tf.random.normal((batch_size, seq_len, self.latent_dim))
        decoder_input = self._get_decoder_input(z, labels)
        return (self.decoder(decoder_input), labels)
    
    def call(self, data):
        X, labels = data
        encoder_input = self._get_encoder_input(X, labels)
        z_mean, _, _ = self.encoder(encoder_input)
        
        decoder_input = self._get_decoder_input(z_mean, labels)
        x_decoded = self.decoder(decoder_input)
        if len(x_decoded.shape) == 1:
            x_decoded = x_decoded.reshape((1, -1))
        return x_decoded
    
    def _get_reconstruction_loss(self, X, Xr):
        reconst_loss = tf.reduce_sum(tf.math.squared_difference(X, Xr)) +\
                       tf.reduce_sum(tf.math.squared_difference(tf.reduce_mean(X, axis=1), tf.reduce_mean(Xr, axis=1))) +\
                       tf.reduce_sum(tf.math.squared_difference(tf.reduce_mean(X, axis=2), tf.reduce_mean(Xr, axis=2)))
        return reconst_loss
    
    def _get_encoder_input(self, X, labels):
        if self._temporal:
            return tf.concat([X, labels[:, :, None]], axis=2)
        else:
            rep_labels = tf.repeat(labels[:, None, :], [seq_len], axis=1)
            return tf.concat([X, rep_labels], axis=2)
    
    def _get_decoder_input(self, z, labels):
        if self._temporal:
            rep_labels = labels[:, :, None]
        else:
            rep_labels = tf.repeat(labels[:, None, :], [seq_len], axis=1)
        z = tf.reshape(z, [-1, seq_len, latent_dim])
        return tf.concat([z, rep_labels], axis=2)

    def train_step(self, data):
        X, labels = data
        with tf.GradientTape() as tape:
            encoder_input = self._get_encoder_input(X, labels)
            z_mean, z_log_var, z = self.encoder(encoder_input)
            
            decoder_input = self._get_decoder_input(z_mean, labels)
            reconstruction = self.decoder(decoder_input)
            reconstruction_loss = self._get_reconstruction_loss(X, reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + self.beta * kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [ ]:
architecture = cVAE_CONV5Architecture(seq_len=seq_len, feat_dim=feat_dim, latent_dim=latent_dim)

encoder, decoder = architecture._encoder, architecture._decoder

In [ ]:
encoder.summary()

In [ ]:
decoder.summary()

In [ ]:
X, y_i = tsgm.utils.gen_sine_vs_const_dataset(5000, 256, 1, max_value=20, const=10)

scaler = tsgm.utils.TSFeatureWiseScaler((0, 1))
X = scaler.fit_transform(X).astype(np.float32)
y = keras.utils.to_categorical(y_i, num_classes).astype(np.float32)

In [ ]:
encoder, decoder = architecture._encoder, architecture._decoder

vae = cBetaVAE(encoder, decoder, latent_dim=latent_dim, temporal=False)
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0003,
    decay_steps=100,
    decay_rate=0.9)
optimizer = keras.optimizers.Adam(lr_schedule)
vae.compile(optimizer=keras.optimizers.Adam())

vae.fit(X, y, epochs=1000, batch_size=128)

In [ ]:
x_decoded = vae.predict([X, y])

In [ ]:
limit = 1000

In [ ]:
X_gen, y_gen = vae.generate(y[:limit])
X_gen = X_gen.numpy()

In [ ]:
def visualize_original_and_reconst_ts(original, reconst, num=5, vmin=0, vmax=1):
    assert original.shape == reconst.shape

    fig, axs = plt.subplots(num, 2, figsize=(14, 10))

    ids = np.random.choice(original.shape[0], size=num, replace=False)
    for i, sample_id in enumerate(ids):
        axs[i, 0].imshow(original[sample_id].T, aspect='auto', vmin=vmin, vmax=vmax)
        axs[i, 1].imshow(reconst[sample_id].T, aspect='auto', vmin=vmin, vmax=vmax)

In [ ]:
visualize_original_and_reconst_ts(X, x_decoded, num=10)

## Visualize using TSNE

In [ ]:
import numpy as np
from sklearn.manifold import TSNE

limit = 1000
X_sub, y_sub = X[:limit], y[:limit]
tsne = TSNE(n_components=2, learning_rate='auto',
                  init='random')

X_all = np.concatenate((X_sub, X_gen))
y_all = np.concatenate((y_sub, y_gen))
c = np.argmax(np.concatenate((y_sub, y_gen)), axis=1)
colors = {0: "class 0", 1: "class 1"}
c = [colors[el] for el in c]
point_styles = ["hist"] * X_sub.shape[0] + ["gen"] * X_gen.shape[0]
X_emb = tsne.fit_transform(np.resize(X_all, (X_all.shape[0], X_all.shape[1] * X_all.shape[2])))

In [ ]:
plt.figure(figsize=(8, 6), dpi=80)
sns.scatterplot(x=X_emb[:, 0], y=X_emb[:, 1], hue=c[:], style=point_styles[:], markers={"hist": "<", "gen": "H"}, alpha=0.7)
#sns.scatterplot(x=X_emb[limit:, 0], y=X_emb[limit:, 1], hue=c[limit:], style=point_styles[limit:], marker="s", alpha=0.5)
plt.legend()
plt.box(False)
plt.axis('off')
plt.show()